# Attack Crypter
> An open source travesty used to drop .NET stealers

- toc: true 
- badges: true
- categories: [attack crypter,downloader,dotnet,open source]

## Overview
AttackCrypt is an open source "crypter" project that can be used to "protect" binaries and "prevent" detection by AV. In the words of the developer...
> I don t know how much This will stay FUD but will be updating it always and adding New Injection and new Attacks to it

This crypter has recently been used to "protect" VenomRAT and is actively in use in the wild.

GitHub: [Theattacker-Crypter](https://github.com/TheNewAttacker64/Theattacker-Crypter)

![](https://i.imgur.com/wLEkFW0.png)

## Sample
[11c38fc24bf7b29cd6e974670bc11d7f92af124d8b7edcd89482500f4de3d442](https://www.unpac.me/results/99994980-a37e-4c4e-bc23-4e62da9461fb/#/)

## References 
- [Attacker-Crypter (v0.9): Unveiling a Powerful Tool for Evading Antivirus and Enhancing Malware Capabilities](https://www.cyfirma.com/outofband/attacker-crypter-v0-9-unveiling-a-powerful-tool-for-evading-antivirus-and-enhancing-malware-capabilities/)



## Analysis

The crypter uses a generated mutex to ensure only one version of the payload is installed, but this mutex is not random it is prepended with the string `attackercrypter` lol!

The crypter also claims to have anti-vm checks which are optional... these are simply a *manufactor* check via WMI `Select * from Win32_ComputerSystem`.

```c#
if ((manufacturer == "microsoft corporation" && item["Model"].ToString().ToUpperInvariant().Contains("VIRTUAL"))
                            || manufacturer.Contains("vmware")
                            || item["Model"].ToString() == "VirtualBox")
```

### Payload Decryption

**Spoiler alert this is not really a cryptor lol!** Instead of protecting the payload the tool converts it into encrypted data that is stored as a string. This string must then be uploaded to a "drop site" but the operator and then the "drop site" URL placed in the tool to generate the "stub". This "stub" is not a stub it is actually just a downloader that will fetch the payload, decrypt it, and load it.

This is the opposite of what you would like in a crypter as the loader will actually look the same for every "build". Let's check in on the VT detection rate in a week: [11c38fc24bf7b29cd6e974670bc11d7f92af124d8b7edcd89482500f4de3d442](https://www.virustotal.com/gui/file/11c38fc24bf7b29cd6e974670bc11d7f92af124d8b7edcd89482500f4de3d442).

In this sample the drop site is `http[:]//paste.sensio[.]no/ReplicaSerena`

An example of the "encrypted" data follows.

```
'Y','A','K','F','E','x','y','o','2','y','v','U','B','Z','N','C','q','R','j','O','l','Y'
```

#### Decryption

[Cyber Chef link](https://gchq.github.io/CyberChef/#recipe=Find_/_Replace(%7B'option':'Simple%20string','string':'%5C''%7D,'',true,false,true,false)Find_/_Replace(%7B'option':'Simple%20string','string':','%7D,'',true,false,true,false)Reverse('Character')From_Base64('A-Za-z0-9%2B/%3D',true,false)AES_Decrypt(%7B'option':'Base64','string':'cT0j6Iw9VylE9o8lcfS4/Bcb8loeSeBirgvin5wpiwg%3D'%7D,%7B'option':'Base64','string':'0SRVQvZDd6l4hBTnn%2BE0TQ%3D%3D'%7D,'CBC','Raw','Raw',%7B'option':'Hex','string':''%7D,%7B'option':'Hex','string':''%7D))


- Remove `,` and `'` from the string (deobfuscate)
- Reverse the string
- Base64 decode
- Decrypt with hard coded key and IV (AES CBC)
  - The key and IV are base64 encoded 

In this sample the Key is `cT0j6Iw9VylE9o8lcfS4/Bcb8loeSeBirgvin5wpiwg=` and the IV is `0SRVQvZDd6l4hBTnn+E0TQ==`.

## Yara Rule

```c
rule AttackCrypter {
    strings:
    
        $s1 = "attackercrypter" wide
        $s2 = "$FOLDER" wide
        $s3 = "$FNAME.exe" wide
        $s4 = "$service" wide
        $s5 = "$serverpassword" wide
        $s6 = "$bottoken" wide
        $s7 = "$chatid" wide
        $s8 = "#NATIVEINJECTPATH" wide
        $s9 = "#DOTNETINJECTPATH" wide
        $s10 = "https://api.telegram.org/bot" wide
        $s11 = "Don t use on vm" wide
            
        $m1 = "FOKFILE" ascii
        $m2 = "FOKSTRING" ascii
        $m3 = "NIKBINARY32bit" ascii
        $m4 = "NIKFELSTART" ascii
            
    condition:
        5 of ($s*) and
        any of ($m*)
}
```
